<a href="https://colab.research.google.com/github/rugveddarwhekar/board-games-faq/blob/main/BoardGames_FAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Board Game FAQ Chat App

This project demonstrates how to build a smart chatbot for board games using a Large Language Model (LLM) and implement Retrieval Augmented Generation (RAG) with the LangChain framework. The chatbot reads rulebooks and leverages Google's latest language model namely Google Gemini 1.5 Flash to provide accurate answers to questions about the games.

---

This application currently supports the following games:


* Catan base game & 5-6 player expansion
* Codenames
* Pandemic
* Monopoly
* Ticket to Ride base game & expansions


## Project Setup

### Install dependencies

In [45]:
!pip install -q langchain google-search chromadb pypdf unstructured langchain-community langchain-google-genai

### Import Libraries

In [46]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredFileLoader
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from google.colab import userdata
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display
from IPython.display import Markdown
import textwrap

### Setup Gemini API

In [47]:
GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

## Load Rulebook PDF Files

In [67]:
# Setup the directory where files are stored
pdf_directory = "/content/board_games"

# Create a list to store the PDFs
documents = []

# Iterate through each file in the directory
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        filepath = os.path.join(pdf_directory, filename)
        loader = PyPDFLoader(filepath)
        documents.extend(loader.load())

# Split the documents into smaller chunks for efficient processing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=6000, chunk_overlap=750)
docs = text_splitter.split_documents(documents)

# Create a Chroma vectorstore for efficient document retrieval
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)
vectordb = Chroma.from_documents(docs, embeddings)

## Set Up the LLM and RetrievalQA Chain

In [68]:
# Initialize the Google Gemini 1.5 Flash model
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY, temperature=0.2)

# Get user input and generate a response
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just mention that you we do not support this currently and do no try to make up an answer and mention that we may support it in the future. Always say "Thanks for asking" at the end of the answer.
{context}
Question: {question}
Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vectordb.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

## Questions

Here we will query some sample questions.    
**Note**: Make sure to include the name of the board game since certain terms maybe common to multiple board games.

In [60]:
query = "How to get out of jail in monopoly?"
result = qa_chain({"query": query})
Markdown(result["result"])

You can get out of jail in Monopoly by:

1. **Throwing doubles on any of your next three turns:** If you succeed, you immediately move forward the number of spaces shown by your doubles throw. You do not get another turn.
2. **Using the "Get Out of Jail Free" card:** If you have one, you can play it to get out of jail.
3. **Purchasing the "Get Out of Jail Free" card from another player:** You can buy it from another player and play it to get out of jail.
4. **Paying a fine of $50:** You can pay this fine before you roll the dice on either of your next two turns.

If you do not throw doubles by your third turn, you must pay the $50 fine. You then get out of jail and immediately move forward the number of spaces shown by your throw.

Thanks for asking. 


In [61]:
query = "What is the maximum number of cards a player can have in their hand in Pandemic?"
result = qa_chain({"query": query})
Markdown(result["result"])

The maximum number of cards a player can have in their hand in Pandemic is **7**. 

Thanks for asking. 


In [62]:
query = "How many destination cards do you get to keep in Ticket to Ride?"
result = qa_chain({"query": query})
Markdown(result["result"])

The number of destination cards you must keep varies depending on the game you are playing. Here are the rules for each version:

* **USA:** You must keep at least 2 destination cards out of the 3 you are dealt at the start of the game. You can keep all 3 if you wish.
* **USA + 1910 Expansion:**  You must keep at least 2 destination cards out of the 3 you are dealt at the start of the game. You can keep all 3 if you wish.
* **Europe:** You must keep at least 2 destination cards out of the 4 you are dealt at the start of the game (3 regular and 1 long route card).
* **Europe + 1912 Expansion:**  You must keep at least 2 destination cards out of the 4 you are dealt at the start of the game (3 regular and 1 long route card).  
* **Mega Europe:** You must keep at least 2 destination cards if you kept a long route card, or 3 if you didn't. You are dealt 5 destination cards at the start of the game.
* **Big Cities of Europe:** You must keep at least 2 destination cards out of the 5 you are dealt at the start of the game.
* **Mega Game:** You must keep at least 3 destination cards out of the 5 you are dealt at the start of the game.

Thanks for asking! 


In [70]:
query = "How does the Star Wars game begin?"
result = qa_chain({"query": query})
Markdown(result["result"])

The provided text is about the game Settlers of Catan, not Star Wars. We do not support answering questions about Star Wars at this time. We may support this in the future. Thanks for asking. 
